＃ ElasticSearch

> [Elasticsearch](https://www.elastic.co/elasticsearch/)是一个分布式的、基于REST的搜索和分析引擎。它提供了一个分布式的、多租户能力的全文搜索引擎，带有HTTP Web界面和无模式JSON文档。

本笔记本展示了如何使用与'Elasticsearch'数据库相关的功能。

＃ ElasticVectorSearch类

## 安装

查看[Elasticsearch安装说明] （https://www.elastic.co/guide/en/elasticsearch/reference/current/install-elasticsearch.html）。

要连接到不需要登录凭据的Elasticsearch实例，请将Elasticsearch URL和索引名称以及嵌入对象传递给构造函数。

例如：
```python
from langchain import ElasticVectorSearch
from langchain.embeddings import OpenAIEmbeddings

embedding = OpenAIEmbeddings()
elastic_vector_search = ElasticVectorSearch(
elasticsearch_url =“http://localhost:9200”，
index_name =“test_index”，
embedding =嵌入
)
```

要连接到需要登录凭据的Elasticsearch实例（包括Elastic Cloud），请使用Elasticsearch URL格式https://username：password @ es_host:9243。例如，要连接到弹性云，请使用所需的身份验证详细信息创建Elasticsearch URL，并将其作为命名参数elasticsearch_url传递给ElasticVectorSearch构造函数。

您可以通过登录Elastic Cloud控制台（https://cloud.elastic.co），选择部署并导航到“部署”页来获取Elastic Cloud URL和登录凭据。

要获取默认“elastic”用户的Elastic Cloud密码：
1.登录https://cloud.elastic.coElastic Cloud控制台
2.前往“安全性”>“用户”
3.找到“elastic”用户并单击“编辑”
4.单击“重置密码”
5.按提示重置密码

Elastic Cloud URL的格式为https：// username：password @ cluster_id.region_id.gcp.cloud.es.io:9243。

例如：
```python
from langchain import ElasticVectorSearch
from langchain.embeddings import OpenAIEmbeddings

embedding = OpenAIEmbeddings()

elastic_host =“cluster_id.region_id.gcp.cloud.es.io”
elasticsearch_url = f”https://username:password@{elastic_host}:9243”
elastic_vector_search = ElasticVectorSearch(
elasticsearch_url = elasticsearch_url，
index_name =“test_index”，
embedding =嵌入
)
```

In [ ]:
!pip install elasticsearch

In [ ]:
import os
import getpass

os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')

OpenAI API Key: ········


## 示例

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import ElasticVectorSearch
from langchain.document_loaders import TextLoader

In [ ]:
from langchain.document_loaders import TextLoader
loader = TextLoader('../../../state_of_the_union.txt')
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

In [ ]:
db = ElasticVectorSearch.from_documents(docs, embeddings, elasticsearch_url="http://localhost:9200")

query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query)

In [ ]:
print(docs[0].page_content)

In state after state, new laws have been passed, not only to suppress the vote, but to subvert entire elections. 

We cannot let this happen. 

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


＃ ElasticKnnSearch类
‘ElasticKnnSearch’实现了允许在Elasticsearch中存储向量和文档以供使用近似[kNN搜索]（https：//www.elastic.co/guide/en/elasticsearch/reference/current/knn-search.html）的特性

In [ ]:
!pip install langchain elasticsearch

In [ ]:
from langchain.vectorstores.elastic_vector_search import ElasticKnnSearch
from langchain.embeddings import ElasticsearchEmbeddings
import elasticsearch

In [ ]:
# Initialize ElasticsearchEmbeddings
model_id = "<model_id_from_es>" 
dims = dim_count
es_cloud_id = "ESS_CLOUD_ID"
es_user = "es_user"
es_password = "es_pass"
test_index = "<index_name>"
#input_field = "your_input_field" # if different from 'text_field'

In [ ]:
# Generate embedding object
embeddings = ElasticsearchEmbeddings.from_credentials(
    model_id,
    #input_field=input_field,
    es_cloud_id=es_cloud_id,
    es_user=es_user,
    es_password=es_password,
)

In [ ]:
# Initialize ElasticKnnSearch
knn_search = ElasticKnnSearch(
	es_cloud_id=es_cloud_id, 
	es_user=es_user, 
	es_password=es_password, 
	index_name= test_index, 
	embedding= embeddings
)

## 测试添加向量

In [ ]:
# Test `add_texts` method
texts = ["Hello, world!", "Machine learning is fun.", "I love Python."]
knn_search.add_texts(texts)

# Test `from_texts` method
new_texts = ["This is a new text.", "Elasticsearch is powerful.", "Python is great for data analysis."]
knn_search.from_texts(new_texts, dims=dims)

## 使用查询向量生成器测试knn搜索

In [ ]:
# Test `knn_search` method with model_id and query_text
query = "Hello"
knn_result = knn_search.knn_search(query = query, model_id= model_id, k=2)
print(f"kNN search results for query '{query}': {knn_result}")
print(f"The 'text' field value from the top hit is: '{knn_result['hits']['hits'][0]['_source']['text']}'")

# Test `hybrid_search` method
query = "Hello"
hybrid_result = knn_search.knn_hybrid_search(query = query, model_id= model_id, k=2)
print(f"Hybrid search results for query '{query}': {hybrid_result}")
print(f"The 'text' field value from the top hit is: '{hybrid_result['hits']['hits'][0]['_source']['text']}'")

## 使用预生成的向量测试knn搜索


In [ ]:
# Generate embedding for tests
query_text = 'Hello'
query_embedding = embeddings.embed_query(query_text)
print(f"Length of embedding: {len(query_embedding)}\nFirst two items in embedding: {query_embedding[:2]}")

# Test knn Search
knn_result = knn_search.knn_search(query_vector = query_embedding, k=2)
print(f"The 'text' field value from the top hit is: '{knn_result['hits']['hits'][0]['_source']['text']}'")

# Test hybrid search - Requires both query_text and query_vector
knn_result = knn_search.knn_hybrid_search(query_vector = query_embedding, query=query_text, k=2)
print(f"The 'text' field value from the top hit is: '{knn_result['hits']['hits'][0]['_source']['text']}'")

## 测试源选项

In [ ]:
# Test `knn_search` method with model_id and query_text
query = "Hello"
knn_result = knn_search.knn_search(query = query, model_id= model_id, k=2, source=False)
assert not '_source' in knn_result['hits']['hits'][0].keys()

# Test `hybrid_search` method
query = "Hello"
hybrid_result = knn_search.knn_hybrid_search(query = query, model_id= model_id, k=2, source=False)
assert not '_source' in hybrid_result['hits']['hits'][0].keys()

## 测试字段选项

In [ ]:
# Test `knn_search` method with model_id and query_text
query = "Hello"
knn_result = knn_search.knn_search(query = query, model_id= model_id, k=2, fields=['text'])
assert 'text' in knn_result['hits']['hits'][0]['fields'].keys()

# Test `hybrid_search` method
query = "Hello"
hybrid_result = knn_search.knn_hybrid_search(query = query, model_id= model_id, k=2, fields=['text'])
assert 'text' in hybrid_result['hits']['hits'][0]['fields'].keys()

### 测试es客户端连接而不是cloud_id

In [ ]:
# Create Elasticsearch connection
es_connection = Elasticsearch(
    hosts=['https://es_cluster_url:port'], 
    basic_auth=('user', 'password')
)

In [ ]:
# Instantiate ElasticsearchEmbeddings using es_connection
embeddings = ElasticsearchEmbeddings.from_es_connection(
    model_id,
    es_connection,
)

In [ ]:
# Initialize ElasticKnnSearch
knn_search = ElasticKnnSearch(
	es_connection = es_connection,
	index_name= test_index, 
	embedding= embeddings
)

In [ ]:
# Test `knn_search` method with model_id and query_text
query = "Hello"
knn_result = knn_search.knn_search(query = query, model_id= model_id, k=2)
print(f"kNN search results for query '{query}': {knn_result}")
print(f"The 'text' field value from the top hit is: '{knn_result['hits']['hits'][0]['_source']['text']}'")
